In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

from keras.optimizers import Adam

In [ ]:
# Load data
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

# df1 = df1.drop(columns=['sub_area'])
# df2 = df2.drop(columns=['sub_area','row ID'])
df2 = df2.drop(columns=['row ID'])

df1_encoded = pd.get_dummies(df1)
df2_encoded = pd.get_dummies(df2)

X = df1_encoded.drop(columns=['price_doc'], axis=1)
y = df1_encoded['price_doc']


In [ ]:
# Scale data
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
df2_encoded_scaled = scaler.fit_transform(df2_encoded)

X_scaled = X_scaled.astype('float32')
df2_encoded_scaled = df2_encoded_scaled.astype('float32')


In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)



In [ ]:
# Define the model with dropout
n_features = X_scaled.shape[1]
model = Sequential()
model.add(Dense(150, input_dim=n_features, activation='relu'))
model.add(Dropout(0.2))  # Add dropout with a rate of 0.5
model.add(Dense(80, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(35, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [ ]:
# Compile the model with a different optimizer
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Create an Adam optimizer with custom parameters
custom_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7)

# Compile the model with the custom optimizer
model.compile(optimizer=custom_optimizer, loss='mean_squared_error', metrics=['mse'])

In [ ]:
# Print model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               28800     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dropout_2 (Dropout)         (None, 25)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 2

In [ ]:
# Fit the model
model.fit(X_train, y_train, epochs=100, batch_size=256, validation_data=(X_test, y_test))


Epoch 1/100


568/568 [==============================] - 4s 6ms/step - loss: 652353120239616.0000 - mse: 652353120239616.0000 - val_loss: 495170873196544.0000 - val_mse: 495170873196544.0000
Epoch 2/100
568/568 [==============================] - 3s 5ms/step - loss: 325867523801088.0000 - mse: 325867523801088.0000 - val_loss: 240845240401920.0000 - val_mse: 240845240401920.0000
Epoch 3/100
568/568 [==============================] - 3s 5ms/step - loss: 267251487866880.0000 - mse: 267251487866880.0000 - val_loss: 217177353355264.0000 - val_mse: 217177353355264.0000
Epoch 4/100
568/568 [==============================] - 3s 5ms/step - loss: 253934069350400.0000 - mse: 253934069350400.0000 - val_loss: 206271995183104.0000 - val_mse: 206271995183104.0000
Epoch 5/100
568/568 [==============================] - 3s 5ms/step - loss: 245816899928064.0000 - mse: 245816899928064.0000 - val_loss: 200299943821312.0000 - val_mse: 200299943821312.0000
Epoch 6/100
568/568 [==============================] 

In [ ]:

# Predictions
pred = model.predict(df2_encoded_scaled)

# Save predictions to the sample_submission file
df_sample = pd.read_csv("sample_submission.csv")
df_sample['price_doc'] = pred
df_sample.to_csv("sample_submission.csv", index=False)

2431/2431 [==============================] - 3s 1ms/step
